In [1]:
import glob
import tensorflow as tf
cat_filenames = glob.glob('./train/cat*.jpg')
dog_filenames = glob.glob('./train/dog*.jpg')
image_filenames = glob.glob('./train/*.jpg')

In [17]:
len(image)

2000

In [4]:
def get_data(filenames):
    sess = tf.Session()
    label = []
    data = []
    for name in filenames:
        label.append(name.split('/')[2].split('.')[0])
        image_file = tf.read_file(name)
        image = tf.image.decode_jpeg(image_file)
        image = tf.image.resize_images(image,(150,150))
        i = image.eval(session = sess)
        data.append(i)
    return data,label
cat_images,cat_labels = get_data(cat_filenames[:1000])
dog_images,dog_labels = get_data(dog_filenames[:1000])

In [5]:
image = cat_images+dog_images
label = cat_labels+dog_labels

In [6]:
import pandas as pd
import numpy as np
def format_label(labels):
    l=pd.DataFrame(labels,columns=['labels'])
    label = pd.get_dummies(l)
    return np.asarray(label)
def format_image(images):
    return (np.asarray(images))/255
def format_bilabel(labels):
    return np.asarray([1 if x == 'cat' else 0 for x in labels])
    

In [7]:
labels = format_label(label)
images = format_image(image)

images_low= format_image(image_low)
labels_low = format_label(label_low)

In [15]:
def get_input(images):
    return tf.placeholder(tf.float32,[None,150,150,3])
def get_labels(labels):
    return tf.placeholder(tf.float32,[None,2])
def get_prob():
    return tf.placeholder(tf.float32,None)

In [16]:
def conv_pool(inputs,conv_ksize,conv_stride,pool_ksize,pool_stride,n_output):
    w = tf.Variable(tf.truncated_normal([conv_ksize,
                                         conv_ksize,
                                         inputs.get_shape().as_list()[3],
                                         n_output],mean=0.0, stddev=0.05,dtype = tf.float32))
    b = tf.Variable(tf.constant(0.0,shape = [n_output],dtype = tf.float32),trainable = True)
    conv = tf.nn.conv2d(inputs,w,[1,conv_stride,conv_stride,1],padding = 'SAME')
    conv = tf.nn.bias_add(conv,b)
    conv = tf.nn.relu(conv)
    pool = tf.nn.max_pool(conv,[1,pool_ksize,pool_ksize,1],[1,pool_stride,pool_stride,1],'SAME')
    return pool


In [17]:
def flatten(inputs):
    return tf.contrib.layers.flatten(inputs)

In [18]:
def fully_connected(inputs,n_output):
    w = tf.Variable(tf.truncated_normal([inputs.get_shape().as_list()[1],n_output],mean=0, stddev=5e-2))
    b = tf.Variable(tf.constant(1.0,shape =[n_output],dtype = tf.float32),trainable = True)
    l = tf.nn.relu(tf.add(tf.matmul(inputs,w),b))
    return l

In [19]:
def output_layer(inputs,n_output):
    w = tf.Variable(tf.truncated_normal([inputs.get_shape().as_list()[1],n_output],mean=0, stddev=5e-2))
    b = tf.Variable(tf.constant(1.0,shape = [n_output],dtype = tf.float32),trainable = True)
    return tf.nn.softmax(tf.add(tf.matmul(inputs,w),b))

In [20]:
def conv_net(inputs,keep_prob):
    conv_ksize = 3
    conv_stride = 1
    pool_ksize = 2
    pool_stride = 2
    n_output = 32
    a = conv_pool(inputs,conv_ksize,conv_stride,pool_ksize,pool_stride,n_output)
    a = tf.nn.dropout(a,keep_prob)
    a = conv_pool(a,conv_ksize,conv_stride,pool_ksize,pool_stride,32)
    a = tf.nn.dropout(a,keep_prob)
    a = conv_pool(a,conv_ksize,conv_stride,pool_ksize,pool_stride,64)
    a = tf.nn.dropout(a,keep_prob)
    a = conv_pool(a,conv_ksize,conv_stride,pool_ksize,pool_stride,64)
    a = tf.nn.dropout(a,keep_prob)
    a = flatten(a)
#    a = fully_connected(a,1000)
    a = fully_connected(a,64)
    a = tf.nn.dropout(a,keep_prob)
    a = fully_connected(a,32)
    a = output_layer(a,2)
    return a 

train_l = pd.DataFrame(train_labels,columns = ['animal'])
train_l = pd.get_dummies(train_l.animal)
train_l = np.asarray(train_l)
train_l.shape

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [22]:
inputs = get_input(images)
labels = get_labels(labels)
keep_prob = get_prob()
logits = conv_net(inputs,keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                              labels = labels))
optimizer = tf.train.RMSPropOptimizer(0.0001).minimize(cost)
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(logits),1),tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
def train_network(sess,optimizer,keep_prob, data, label):
    sess.run(optimizer,feed_dict = {inputs:data,labels:label,keep_prob:0.5})
def print_stats(session, feature_batch, label_batch, cost, accuracy):

    loss = session.run(cost, feed_dict={inputs:feature_batch,labels:label_batch,keep_prob:1})
    acc = session.run(accuracy,feed_dict = {inputs:feature_batch,labels:label_batch,keep_prob:1})
    validation_acc = session.run(accuracy,feed_dict = {inputs:X_test,labels:y_test,keep_prob:1})
    print ('loss = ',loss,'training accuracy =',acc,  'validation accuracy = ',validation_acc)

In [25]:
epochs =8
batch_size = 16
n_steps =1001
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print ("Learning Starts:")
    for epoch in range(epochs):
        for step in range(n_steps):
            start = (step*batch_size)%(y_train.shape[0]-batch_size)
            batch_data = X_train[start:(start+batch_size),:,:,:]
            batch_label = y_train[start:(start+batch_size),:]
            train_network(sess, optimizer, keep_prob, batch_data, batch_label)
            if (step % 50 == 0):
                print('Epoch {:>2}'.format(epoch + 1), end='')
                print('  Step {:>2}'.format(step +1), end = '')
                print_stats(sess, batch_data, batch_label, cost, accuracy)
    #print (sess.run(logits,feed_dict = {inputs:X_train[4996:5004],keep_prob:1}))


Learning Starts:
Epoch  1  Step  1loss =  0.694331 training accuracy = 0.375 validation accuracy =  0.5025
Epoch  1  Step 51loss =  0.692786 training accuracy = 0.5 validation accuracy =  0.4975
Epoch  1  Step 101loss =  0.692626 training accuracy = 0.5625 validation accuracy =  0.5025
Epoch  1  Step 151loss =  0.688967 training accuracy = 0.625 validation accuracy =  0.4975
Epoch  1  Step 201loss =  0.69273 training accuracy = 0.6875 validation accuracy =  0.5725
Epoch  1  Step 251loss =  0.68988 training accuracy = 0.5625 validation accuracy =  0.4975
Epoch  1  Step 301loss =  0.692187 training accuracy = 0.5 validation accuracy =  0.5
Epoch  1  Step 351loss =  0.702487 training accuracy = 0.3125 validation accuracy =  0.4975
Epoch  1  Step 401loss =  0.681089 training accuracy = 0.8125 validation accuracy =  0.4975
Epoch  1  Step 451loss =  0.690293 training accuracy = 0.5625 validation accuracy =  0.4975
Epoch  1  Step 501loss =  0.685571 training accuracy = 0.625 validation accura